Parallel Processing for best model


In [ ]:
import numpy as np
from scipy.optimize import differential_evolution
from concurrent.futures import ProcessPoolExecutor
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

def read_data():
    with open('/content/data.txt', 'r') as file:
        lines = file.readlines()
    v_values = np.fromstring(lines[0].split(':')[1], dtype=float, sep=' ')
    P_values = np.fromstring(lines[1].split(':')[1], dtype=float, sep=' ')
    mu_data = np.loadtxt(lines[3:])
    return v_values, P_values, mu_data

# Friction models
def friction_coefficient_lupker(vt_pn, *params):
    vt, pn = vt_pn
    p, v, p0, k, mu_s, mu_m, u, v_max, h = params
    exp_term = np.exp(-h**2 * (np.log(u / np.maximum(v_max, 1e-10))**2))
    return ((pn / np.maximum(p0, 1e-10)) ** -k) * (mu_s + (mu_m - mu_s) * exp_term)

def mu_equation_wriggers(vt_pn, *params):
    vt, pn = vt_pn
    c1, c2, c3, c4, c5, c6, c7 = params
    return (c1 * ((pn / np.maximum(c2, 1e-10)) ** c3) + c4 * np.log1p(np.maximum(vt/c5 - 1, 0)) - c6 * np.log1p(np.maximum(vt/c7 - 1, 0)))

def friction_coefficient_dorsh(vt_pn, *params):
    vt, pn = vt_pn
    c1, c2, c3 = params
    return c1 * (pn ** c2) * (vt ** c3)

def huemer_friction_law(vt_pn, *params):
    vt, pn = vt_pn
    a, b, c, n, m, alpha, beta = params
    return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

def friction_coefficient_wriggers_reinelt(vt_pn, *params):
    vt, pn = vt_pn
    a, b, c, d = params
    v2 = vt**2 + (a * pn)**2
    return (2 * vt * a * pn / np.maximum(v2, 1e-10))**c * (b / np.maximum(pn, 1e-10)) * np.arctan(d * pn)

# Objective functions defined explicitly at the top level
def objective_function_lupker(params, VT, PN, mu):
    return np.mean(np.abs(friction_coefficient_lupker((VT.flatten(), PN.flatten()), *params) - mu) / mu)

def objective_function_wriggers(params, VT, PN, mu):
    return np.mean(np.abs(mu_equation_wriggers((VT.flatten(), PN.flatten()), *params) - mu) / mu)

def objective_function_dorsh(params, VT, PN, mu):
    return np.mean(np.abs(friction_coefficient_dorsh((VT.flatten(), PN.flatten()), *params) - mu))

def objective_function_huemer(params, VT, PN, mu):
    return np.mean(np.abs((huemer_friction_law((VT.flatten(), PN.flatten()), *params) - mu) / mu) * 100)

def objective_function_wriggers_reinelt(params, VT, PN, mu):
    return np.mean(np.abs((friction_coefficient_wriggers_reinelt((VT.flatten(), PN.flatten()), *params) - mu) / mu))

# Models configuration and execution
models = [
    {
        "name": "Lupker Model",
        "model": friction_coefficient_lupker,
        "objective_function": objective_function_lupker,
        "bounds": [(0.1, 100.0), (0.1, 100.0), (0.1, 2), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0)]
    },
    {
        "name": "Wriggers Model",
        "model": mu_equation_wriggers,
        "objective_function": objective_function_wriggers,
        "bounds": [(0.1, 20), (0.1, 20), (-1.0, 1.0), (0.1, 10.0), (0.1, 20), (0.1, 10.0), (0.1, 1.0)]
    },
    {
        "name": "Dorsh Model",
        "model": friction_coefficient_dorsh,
        "objective_function": objective_function_dorsh,
        "bounds": [(0.1, 2), (-2, 2), (-2, 2)]
    },
    {
        "name": "Huemer Model",
        "model": huemer_friction_law,
        "objective_function": objective_function_huemer,
        "bounds": [(1.0, 10.0), (-10.0, 1.0), (1.0, 10.0), (1.0, 10.0), (1.0, 10.0), (-10.0, -1.0), (1.0, 10.0)]
    },
    {
        "name": "Wriggers and Reinelt Model",
        "model": friction_coefficient_wriggers_reinelt,
        "objective_function": objective_function_wriggers_reinelt,
        "bounds": [(0, 1), (0, 1), (-1, 0), (1, 2)]
    }
]

def optimize_and_execute(model, objective_function, bounds, vt_input, pn_input, v_values, P_values, mu_data):
    VT, PN = np.meshgrid(v_values, P_values)
    mu = mu_data.T.ravel()

    # Perform optimization using differential evolution
    result = differential_evolution(objective_function, bounds, args=(VT, PN, mu), strategy='best1bin', popsize=15, tol=1e-4)
    params = result.x

    # Calculate mu using the optimized parameters
    calculated_mu = model((vt_input, pn_input), *params)

    # Determine the actual mu from the data for the nearest v and p values
    vt_index = np.argmin(np.abs(v_values - vt_input))
    pn_index = np.argmin(np.abs(P_values - pn_input))
    actual_mu = mu_data[vt_index, pn_index]

    # Calculate the Mean Absolute Percentage Error (MAPE)
    mape = np.abs((calculated_mu - actual_mu) / actual_mu) * 100

    return calculated_mu, mape, params

# Ensure the main function is correct
def main():
    v_values, P_values, mu_data = read_data()
    vt_input = float(input("Enter the value for VT (mm/s): "))
    pn_input = float(input("Enter the value for PN (MPa): "))

    with ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(optimize_and_execute, model["model"], model["objective_function"], model["bounds"], vt_input, pn_input, v_values, P_values, mu_data)
            for model in models
        ]
        results = [(future.result(), model["name"]) for future, model in zip(futures, models)]

    best_model = min(results, key=lambda x: x[0][1])
    print(f"The best model is {best_model[1]} with a MAPE of {best_model[0][1]:.2f}% and a calculated mu of {best_model[0][0]:.4f}")
    print("Optimal parameters for the best model:", best_model[0][2])

if __name__ == "__main__":
    main()



Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0
The best model is Huemer Model with a MAPE of 0.00% and a calculated mu of 0.5569
Optimal parameters for the best model: [ 1.00346154 -1.11822542  1.22815888  1.04060779  9.98376168 -3.93310115
  4.46386637]


Choosing a model based on USER preference

In [2]:
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import minimize

def get_user_input():
    vt_input = float(input("\nEnter the value for VT (mm/s): "))
    pn_input = float(input("Enter the value for PN (MPa): "))
    return vt_input, pn_input
# Define the Lupker model
def lupker_model(vt_input,pn_input):

    # Function to read data from external file
    def read_data(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
        v_values = np.fromstring(lines[0].split(':')[1], dtype=float, sep=' ')
        P_values = np.fromstring(lines[1].split(':')[1], dtype=float, sep=' ')
        mu_data = np.loadtxt(lines[3:])
        return v_values, P_values, mu_data

    # Function to calculate friction coefficient
    def friction_coefficient(vt, pn, p, v, p0, k, mu_s, mu_m, u, v_max, h):
        return ((pn / p0) ** -k) * (mu_s + (mu_m - mu_s) * np.exp(-h**2 * (np.log(u / v_max)**2)))

    # Function to define objective function
    def objective_function(params, vt, pn, mu):
        p, v = params[:2]
        other_params = params[2:]
        mu_calc = friction_coefficient(vt, pn, p, v, *other_params)
        absolute_percentage_error = np.abs((mu_calc - mu) / mu)
        return np.mean(absolute_percentage_error)

    # Function to perform optimization
    def optimize_parameters(v_values, P_values, mu_data, bounds):
        VT, PN = np.meshgrid(v_values, P_values)
        mu = mu_data.T.ravel()
        result = differential_evolution(objective_function, bounds, args=(VT.flatten(), PN.flatten(), mu),
                                        strategy='best1bin', popsize=20, tol=0.001, mutation=(0.5, 1), recombination=0.7)
        return result.x, result.fun

    # Function to get user input for VT and PN

    # Function to calculate friction coefficient for user input
    def calculate_friction_coefficient(vt_input, pn_input, params):
        calculated_mu_input = friction_coefficient(vt_input, pn_input, *params)
        return calculated_mu_input

    # Function to calculate MAPE between calculated and actual mu values
    def calculate_mape(calculated_mu_input, actual_mu):
        mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100
        return mape

    # Function to print the friction coefficients for all combinations of VT and PN using the optimized parameters
    def print_friction_coefficients(v_values, P_values, params):
        print("\nFriction coefficients for all combinations of VT and PN:")
        for i in range(len(v_values)):
            for j in range(len(P_values)):
                calculated_mu = friction_coefficient(v_values[i], P_values[j], *params)
                print("VT={:.1f} mm/s, PN={:.1f} MPa: {:.4f}".format(v_values[i], P_values[j], calculated_mu))

    # Function to calculate the overall MAPE between actual and calculated mu values
    def calculate_overall_mape(mu_data, v_values, P_values, params):
        calculated_mu_data = np.zeros_like(mu_data)
        for i in range(len(v_values)):
            for j in range(len(P_values)):
                calculated_mu_data[i, j] = friction_coefficient(v_values[i], P_values[j], *params)
        absolute_percentage_error = np.abs((calculated_mu_data - mu_data) / mu_data)
        overall_mape = np.mean(absolute_percentage_error) * 100
        return overall_mape

    # Main function
    def main():
        # Read data from external file
        v_values, P_values, mu_data = read_data('/content/data.txt')

        # Define bounds for parameters
        bounds = [(0.1, 100.0), (0.1, 100.0), (0.1, 2), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0)]

        # Perform optimization
        params, mape = optimize_parameters(v_values, P_values, mu_data, bounds)

        # Print the optimal parameter values
        print("\nOptimal parameter values after Differential Evolution optimization:")
        print("p = {:.4f}, v = {:.4f}, p0 = {:.4f}, k = {:.4f}, mu_s = {:.4f}, mu_m = {:.4f}, u = {:.4f}, v_max = {:.4f}, h = {:.4f}".format(*params))

        # Print the MAPE with optimized parameters
        print("Mean Absolute Percentage Error (MAPE) with optimized parameters: {:.4f}%".format(mape))

        # Calculate friction coefficient for user input using optimized parameters
        calculated_mu_input = calculate_friction_coefficient(vt_input, pn_input, params)
        print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

        # Calculate MAPE between calculated and actual mu values
        actual_mu = mu_data[np.abs(v_values - vt_input).argmin(), np.abs(P_values - pn_input).argmin()]
        mape = calculate_mape(calculated_mu_input, actual_mu)

      # Print the friction coefficients for all combinations of VT and PN using the optimized parameters
        print_friction_coefficients(v_values, P_values, params)

        # Calculate and print the overall MAPE between actual and calculated mu values
        overall_mape = calculate_overall_mape(mu_data, v_values, P_values, params)
        print("Overall Mean Absolute Percentage Error (MAPE) between actual and calculated mu values: {:.4f}%".format(overall_mape))

    if __name__ == "__main__":
        main()


# Define the Wrigger model
def wrigger_model(vt_input,pn_input):
        # Define the mu equation
    def mu_equation(vt_pn, c1, c2, c3, c4, c5, c6, c7):
        vt, pn = vt_pn
        return (c1 * ((pn / c2) ** c3) + c4 * np.log(vt / c5) - c6 * np.log(vt / c7))

    # Read data from external file
    def read_data(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
        v_values = np.fromstring(lines[0].split(':')[1], dtype=float, sep=' ')
        P_values = np.fromstring(lines[1].split(':')[1], dtype=float, sep=' ')
        mu_data = np.loadtxt(lines[3:])
        return v_values, P_values, mu_data

    # Define the objective function for differential evolution
    def objective_function(params, vt_pn, mu_data):
        mu_calc = mu_equation(vt_pn, *params)
        absolute_percentage_error = np.abs((mu_calc - mu_data) / mu_data)
        return np.mean(absolute_percentage_error)

    # Main function
    def main():
        # Read data from external file
        v_values, P_values, mu_data = read_data('/content/data.txt')

        # Create meshgrid of VT and PN
        VT, PN = np.meshgrid(v_values, P_values)

        # Prepare the dependent variable (mu)
        mu = mu_data.T.ravel()

        # Define the bounds for parameters
        bounds = [(0.1, 20), (0.1, 20), (-1.0, 1.0), (0.1, 10.0), (0.1, 20), (0.1, 10.0), (0.1, 1.0)]

        # Perform optimization using Differential Evolution
        np.random.seed(42)  # Setting random seed for reproducibility
        result = differential_evolution(objective_function, bounds, args=(np.vstack((VT.flatten(), PN.flatten())), mu), strategy='best1bin', popsize=50, tol=0.01)

        # Extract optimal parameter values
        popt = result.x

        # Print the optimal parameter values
        print("\nOptimal parameter values after Differential Evolution optimization:")
        print("c1 = {:.4f}, c2 = {:.4f}, c3 = {:.4f}, c4 = {:.4f}, c5 = {:.4f}, c6 = {:.4f}, c7 = {:.4f}".format(*popt))


        # Calculate the friction coefficient for the input VT and PN using optimized parameters
        calculated_mu_input = mu_equation((vt_input, pn_input), *popt)
        print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

        # Find the index of the closest VT and PN values in the data
        vt_index = np.argmin(np.abs(v_values - vt_input))
        pn_index = np.argmin(np.abs(P_values - pn_input))

        # Get the corresponding mu value from the data
        actual_mu = mu_data[vt_index, pn_index]

        # Calculate MAPE between calculated and actual mu values
        mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100

        # Print the friction coefficients for all combinations of VT and PN using the optimized parameters
        print("\nFriction coefficients for all combinations of VT and PN:")
        for i in range(len(v_values)):
            for j in range(len(P_values)):
                calculated_mu = mu_equation((v_values[i], P_values[j]), *popt)
                print("VT={:.1f} mm/s, PN={:.1f} MPa: {:.4f}".format(v_values[i], P_values[j], calculated_mu))

        # Calculate the overall MAPE
        mape_array = np.abs((mu_equation((VT.flatten(), PN.flatten()), *popt) - mu_data.T.ravel()) / mu_data.T.ravel()) * 100
        overall_mape = np.mean(mape_array)

        # Print the overall MAPE
        print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))

    if __name__ == "__main__":
        main()

def dorch_model(vt_input,pn_input):
              # Define the friction coefficient function
        def friction_coefficient(vt_pn, c1, c2, c3):
            vt, pn = vt_pn
            return c1 * (pn ** c2) * (vt ** c3)

        # Read data from external file
        with open('/content/data.txt', 'r') as file:
            lines = file.readlines()

        # Extract v_values, P_values, and mu_data
        v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
        P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
        mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

        # Create meshgrid of VT and PN
        VT, PN = np.meshgrid(v_values, P_values)

        # Prepare the dependent variable (mu)
        mu = mu_data.T.ravel()

        # Define objective function for optimization
        def objective_function(params):
            c1, c2, c3 = params
            mu_calc = friction_coefficient((VT.flatten(), PN.flatten()), c1, c2, c3)
            absolute_error = np.abs(mu_calc - mu)
            return np.mean(absolute_error)

        # Define bounds for parameters based on the observed range in the table values
        bounds = [(0.1, 2), (-2, 2), (-2, 2)]

        # Set the initial guess for parameters
        initial_guess = np.random.uniform([0.1, -2, -2], [2, 2, 2], size=(10, 3))

        # Perform optimization using Differential Evolution with Simulated Annealing
        result = differential_evolution(objective_function, bounds, strategy='best1bin', tol=1e-4, seed=42, init=initial_guess)

        # Extract optimal parameter values
        c1_opt, c2_opt, c3_opt = result.x

        # Print the optimal parameter values
        print("Optimal parameter values after Differential Evolution optimization with Simulated Annealing:")
        print("C1 = {:.4f}, C2 = {:.4f}, C3 = {:.4f}".format(c1_opt, c2_opt, c3_opt))


        # Calculate the friction coefficient for the input VT and PN using optimized parameters
        calculated_mu_input = friction_coefficient((vt_input, pn_input), c1_opt, c2_opt, c3_opt)
        print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

        # Find the index of the closest VT and PN values in the data
        vt_index = np.argmin(np.abs(v_values - vt_input))
        pn_index = np.argmin(np.abs(P_values - pn_input))

        # Get the corresponding mu value from the data
        actual_mu = mu_data[vt_index, pn_index]

        # Calculate MAPE between calculated and actual mu values
        mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100


        # Print all mu values for respective VT and PN
        print("\nCalculated mu values for all combinations of VT and PN:")
        for i, vt in enumerate(v_values):
            for j, pn in enumerate(P_values):
                mu_calc = friction_coefficient((vt, pn), c1_opt, c2_opt, c3_opt)
                print("VT={:.1f} mm/s, PN={:.1f} MPa: Calculated Mu = {:.4f}".format(vt, pn, mu_calc))

        # Calculate MAPE between calculated and actual mu values for all data points
        mape_array = np.abs((friction_coefficient((VT.flatten(), PN.flatten()), c1_opt, c2_opt, c3_opt) - mu_data.T.ravel()) / mu_data.T.ravel()) * 100
        overall_mape = np.mean(mape_array)

        # Print the overall MAPE
        print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))

def huemer_model(vt_input,pn_input):
# Define the function to fit
        def huemer_friction_law(vt_pn, a, b, c, n, m, alpha, beta):
            vt, pn = vt_pn
            return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

        # Read data from external file
        with open('/content/data.txt', 'r') as file:
            lines = file.readlines()

        # Extract v_values, P_values, and mu_data
        v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
        P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
        mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

        # Create meshgrid of VT and PN
        VT, PN = np.meshgrid(v_values, P_values)

        # Prepare the dependent variable (mu)
        mu = mu_data.T.ravel()

        # Define objective function for optimization
        def objective_function(params):
            a, b, c, n, m, alpha, beta = params
            mu_calc = huemer_friction_law((VT.flatten(), PN.flatten()), a, b, c, n, m, alpha, beta)
            absolute_percentage_error = np.abs((mu_calc - mu) / mu) * 100
            return np.mean(absolute_percentage_error)

        # Define bounds for parameters
        bounds = [(1.0, 10.0), (-10.0, 1.0), (1.0, 10.0), (1.0, 10.0), (1.0, 10.0), (-10.0, -1.0), (1.0, 10.0)]

        # Perform optimization using Differential Evolution with bounds and strategy 'best1bin'
        result_de = differential_evolution(objective_function, bounds, strategy='best1bin')

        # Use optimized parameters from Differential Evolution
        popt_de = result_de.x

        # Print the optimal parameter values
        print("Optimal parameter values after optimization:")
        print("a = {:.4f}, b = {:.4f}, c = {:.4f}, n = {:.4f}, m = {:.4f}, alpha = {:.4f}, beta = {:.4f}".format(*popt_de))

        # Perform optimization using Powell method with bounds
        result_powell = minimize(objective_function, x0=popt_de, method='Powell', bounds=bounds)

        # Use optimized parameters from Powell method
        popt_powell = result_powell.x



        # Calculate the friction coefficient for the input VT and PN using optimized parameters
        calculated_mu_input = huemer_friction_law((vt_input, pn_input), *popt_powell)
        print("Friction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

        # Find the index of the closest VT and PN values in the data
        vt_index = np.argmin(np.abs(v_values - vt_input))
        pn_index = np.argmin(np.abs(P_values - pn_input))

        # Get the corresponding mu value from the data
        actual_mu = mu_data[vt_index, pn_index]

        # Calculate MAPE between calculated and actual mu values
        mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100


        # Calculate the friction coefficients for all combinations of VT and PN using the optimized parameters from Powell method
        calculated_mu_powell = huemer_friction_law((VT, PN), *popt_powell)

        # Print the mu values for all combinations of VT and PN
        print("Mu values for all combinations of VT and PN:")
        for i, vt in enumerate(v_values):
            for j, pn in enumerate(P_values):
                print(f"VT={vt} mm/s, PN={pn} MPa: {calculated_mu_powell[j, i]:.4f}")

        # Calculate MAPE for all data points
        mu_calc_powell = huemer_friction_law((VT.flatten(), PN.flatten()), *popt_powell)
        mape_array_powell = np.abs((mu_calc_powell - mu_data.T.ravel()) / mu_data.T.ravel()) * 100

        # Calculate the overall MAPE
        overall_mape_powell = np.mean(mape_array_powell)

        # Print the overall MAPE
        print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points (Powell method): {:.4f}%".format(overall_mape_powell))

def wrigger_and_reinelt(vt_input,pn_input):
      # Function to read data from external file
      def read_data(file_path):
          with open(file_path, 'r') as file:
              lines = file.readlines()
          v_values = np.fromstring(lines[0].split(':')[1], dtype=float, sep=' ')
          P_values = np.fromstring(lines[1].split(':')[1], dtype=float, sep=' ')
          mu_data = np.loadtxt(lines[3:])
          return v_values, P_values, mu_data

      # Function to calculate friction coefficient
      def friction_coefficient(v, p, a, b, c, d):
          mu_max = (2 * v * a * p / (v**2 + (a * p)**2))**c * (b / p) * np.arctan(d * p)
          return mu_max

      # Function to define objective function
      def objective_function(params, vt, pn, mu):
          a, b, c, d = params
          mu_calc = friction_coefficient(vt, pn, a, b, c, d)
          absolute_percentage_error = np.abs((mu_calc - mu) / mu)
          return np.mean(absolute_percentage_error)

      def optimize_parameters(v_values, P_values, mu_data):
          bounds = [(0, 1), (0, 1), (-1, 0), (1, 2)]  # Tighter bounds
          VT, PN = np.meshgrid(v_values, P_values)
          mu = mu_data.T.ravel()
          result = differential_evolution(objective_function, bounds, args=(VT.flatten(), PN.flatten(), mu),
                                          strategy='best1bin', popsize=50, tol=0.0001, mutation=(0.5, 1.9), recombination=0.7)
          return result.x, result.fun


      # Function to calculate friction coefficient for user input using optimized parameters
      def calculate_friction_coefficient(vt_input, pn_input, params):
          calculated_mu_input = friction_coefficient(vt_input, pn_input, *params)
          return calculated_mu_input

      # Function to calculate the overall MAPE between actual and calculated mu values
      def calculate_overall_mape(mu_data, v_values, P_values, params):
          calculated_mu_data = np.zeros_like(mu_data)
          for i in range(len(v_values)):
              for j in range(len(P_values)):
                  calculated_mu_data[i, j] = friction_coefficient(v_values[i], P_values[j], *params)
          absolute_percentage_error = np.abs((calculated_mu_data - mu_data) / mu_data)
          overall_mape = np.mean(absolute_percentage_error) * 100
          return overall_mape

      # Main function
      def main():
          # Read data from external file
          v_values, P_values, mu_data = read_data('/content/data.txt')

          # Perform optimization
          params, mape = optimize_parameters(v_values, P_values, mu_data)

          # Print the optimal parameter values
          print("\nOptimal parameter values after Differential Evolution optimization:")
          print("a = {:.4f}, b = {:.4f}, c = {:.4f}, d = {:.4f}".format(*params))

          # Print the MAPE with optimized parameters
          print("Mean Absolute Percentage Error (MAPE) with optimized parameters: {:.4f}%".format(mape))


          # Calculate friction coefficient for user input using optimized parameters
          calculated_mu_input = calculate_friction_coefficient(vt_input, pn_input, params)
          print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

          # Print all the corresponding mu values for (VT and Pn) from the input table
          print("\nCorresponding mu values for all combinations of VT and PN:")
          for i in range(len(v_values)):
              for j in range(len(P_values)):
                  calculated_mu = friction_coefficient(v_values[i], P_values[j], *params)
                  print("VT={:.1f} mm/s, PN={:.1f} MPa: {:.4f}".format(v_values[i], P_values[j], calculated_mu))

          # Calculate and print the overall MAPE between actual and calculated mu values
          overall_mape = calculate_overall_mape(mu_data, v_values, P_values, params)
          print("\nOverall Mean Absolute Percentage Error (MAPE) between actual and calculated mu values: {:.4f}%".format(overall_mape))

      if __name__ == "__main__":
          main()



# Function to get user input for model choice
def get_model_choice():
    print("Choose a model:")
    print("1. Lupker Model")
    print("2. Wrigger Model")
    print("3. Dorch Model")
    print("4. Huemer Model")
    print("5. WRIGGERS AND REINELT MODEL")
    choice = int(input("Enter your choice: "))
    return choice

# Main function
def main():
    file_path = '/content/data.txt'  # Adjust file path as necessary

    vt_input,pn_input=get_user_input()
    # Get user choice for model
    choice = get_model_choice()

    # Perform operations based on user choice
    if choice == 1:
        lupker_model(vt_input,pn_input)
    elif choice == 2:
        wrigger_model(vt_input,pn_input)
    elif choice == 3:
        dorch_model(vt_input,pn_input)
    elif choice == 4:
        huemer_model(vt_input,pn_input)
    elif choice == 5:
        wrigger_and_reinelt(vt_input,pn_input)
    else:
        print("Invalid choice. Please choose a valid option.")

if __name__ == "__main__":
    main()


Enter the value for VT (mm/s): 100
Enter the value for PN (MPa): 0.5
Choose a model:
1. Lupker Model
2. Wrigger Model
3. Dorch Model
4. Huemer Model
5. WRIGGERS AND REINELT MODEL
Enter your choice: 3
Optimal parameter values after Differential Evolution optimization with Simulated Annealing:
C1 = 0.4935, C2 = -0.2661, C3 = 0.0543

Friction coefficient at VT=100.0 mm/s and PN=0.5 MPa: 0.7619

Calculated mu values for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: Calculated Mu = 0.9107
VT=1.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.5934
VT=1.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.4935
VT=1.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.4430
VT=1.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.4104
VT=10.0 mm/s, PN=0.1 MPa: Calculated Mu = 1.0319
VT=10.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.6724
VT=10.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.5592
VT=10.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.5020
VT=10.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.4650
VT=100.0 mm/s, PN=0.1 MPa: Calculated Mu = 1.1692
VT=100.0 mm/s